# Erstellen und Trainieren der KI

In [1]:
# Needed imports
import numpy as np
from pathlib import Path
from tensorflow import keras

# Needed variabel
dataset_folder = Path(__file__).parent / "dataset"
sequence_length = 64
model_path = Path(__file__).parent / "model.h5"

In [2]:
# Load the dataset
songs = np.load(dataset_folder / "songs.npy")

print(songs.shape)
print(songs[:10])

songs = songs.tolist()

(2576,)
[ 5 16  9 16 16 16 10 16 11 16]


## Erstellen der Trainingsdaten
Wir erstellen sequenzen von eienr bestimmten länge und sagen dem Netzwerk, ok was kommt als nächstes. Dies macht man so lange, bis man alle Daten durchhat. Das schafft man, indem man die die Trainignsdaten nimmt, ein Teil herausschneided und dan den Ausschnit zum Herusschneiden immer eins weiter nach rechts shifted, bis man alle Daten durch ist.

```bash
# Data
[12, 13, 14, 15, 16, 17, ...]

# Erster input und targed
[12, 13] -> 14

#Zweiter input und targed
[13, 14] -> 15
```

In [3]:
# 100 symbols, 64 sl
# 100 - 64 = 36
num_sequences = len(songs) - sequence_length

inputs = []
targets = []

for i in range(num_sequences):
    inputs.append(songs[i:i+sequence_length])
    targets.append(songs[i+sequence_length])

In [4]:
print(f"Num sequences: {len(inputs)}")
print("the first input sequence:")
print(inputs[0])
print("the first target sequence:")
print(targets[0])
print()
print("The whole dataset for the first input sequence:")
print(songs[:sequence_length+1])
print()
print(len(inputs), len(targets))

Num sequences: 2512
the first input sequence:
[5, 16, 9, 16, 16, 16, 10, 16, 11, 16, 16, 10, 9, 16, 13, 16, 16, 16, 16, 16, 10, 16, 16, 16, 12, 16, 11, 16, 16, 10, 9, 16, 8, 16, 16, 16, 9, 16, 10, 16, 16, 16, 16, 16, 17, 16, 16, 16, 5, 16, 13, 16, 16, 16, 13, 16, 11, 16, 16, 10, 9, 16, 15, 16]
the first target sequence:
16

The whole dataset for the first input sequence:
[5, 16, 9, 16, 16, 16, 10, 16, 11, 16, 16, 10, 9, 16, 13, 16, 16, 16, 16, 16, 10, 16, 16, 16, 12, 16, 11, 16, 16, 10, 9, 16, 8, 16, 16, 16, 9, 16, 10, 16, 16, 16, 16, 16, 17, 16, 16, 16, 5, 16, 13, 16, 16, 16, 13, 16, 11, 16, 16, 10, 9, 16, 15, 16, 16]

2512 2512


## One-Hot encoding
Jeder mögliche Wert den es geben kann wird als Reihe in einer tabelle gesehen.
![](./Bilder/oenHot.png)

In unserem Fall ist ist die größe der Tabelle gleich der Anzahl an Elemente in unserem `mapping.json`. In unserem Fall sollte das 18 sein.

In [5]:
print(len(inputs))

2512


In [6]:
vacabulay_size = len(np.unique(songs))
inputs = keras.utils.to_categorical(inputs, num_classes=vacabulay_size)
targets = np.array(targets)

In [7]:
print(f"Vocabulary size: {vacabulay_size}")
print(inputs.shape)
print()
print(inputs)

Vocabulary size: 18
(2512, 64, 18)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 ...

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  

In [8]:
print(targets.shape)
print()
print(targets)

(2512,)

[16 16 16 ...  0  0  0]


## Erstellen des Modells


In [11]:
output_layer_size = vacabulay_size  # Number of unique symbols in the dataset
hidden_layer_sizes = [256]      # Number of neurons in each hidden layer
loss = "sparse_categorical_crossentropy"    # Loss function
activation = "softmax"                      # Activation function
learning_rate = 0.001                       # Learning rate
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)  # Optimizer

Hier erstellt man nun das konkrete Netzwerk.

`shape = (None, output_layer_size)`
1. Variable `None`-> Wie viele Zeitstemnpel hat man. Bei None wird das automatisch ermittelt 
2. Variable `output_layer_size` -> Wie viele Inputs hat man pro Eingabe. Dies ist hier die Anzahl der verwendeten Noten

Dem layer wird dann hidden LSTM layer hinzugefügt und ein Dropout, welches gegen Overfitting hilft.

In [12]:
# create the model achitecture
# model = keras.Sequential() # <- Would be a good idea to use this

# Input layer
input_layer = keras.layers.Input(shape=(None, output_layer_size))

# Hidden layers
x = input_layer
for hidden_layer_size in hidden_layer_sizes:
    x = keras.layers.LSTM(hidden_layer_size)(x)

# Avoid overfitting
x = keras.layers.Dropout(0.3)(x)

# Output layer. Full connection -> Dense
output_layer = keras.layers.Dense(output_layer_size, activation=activation)(x)

# The actual model
model = keras.Model(input_layer, output_layer)

In [13]:
# Compile the model
model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 18)]        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               281600    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 18)                4626      
Total params: 286,226
Trainable params: 286,226
Non-trainable params: 0
_________________________________________________________________


In [15]:
# keras.utils.plot_model(model, 'model.png', show_shapes=True)

## Trainiere das Modell

In [16]:
print(inputs.shape)
print(targets.shape)

shape = inputs[0].shape
for i, x in enumerate(inputs):
    if x.shape != shape:
        print(f"Error: {i}")

(2512, 64, 18)
(2512,)


In [20]:
epochs = 50
batch_size = 64  # TODO Equal to sequence_length ?

result = model.fit(inputs, targets, epochs=epochs)

Epoch 1/50
79/79 [==============================] - 7s 85ms/step - loss: 0.0116 - accuracy: 0.9964
Epoch 2/50
79/79 [==============================] - 7s 88ms/step - loss: 0.0109 - accuracy: 0.9952
Epoch 3/50
79/79 [==============================] - 7s 86ms/step - loss: 0.0098 - accuracy: 0.9964
Epoch 4/50
79/79 [==============================] - 7s 86ms/step - loss: 0.0106 - accuracy: 0.9948
Epoch 5/50
79/79 [==============================] - 7s 85ms/step - loss: 0.0119 - accuracy: 0.9952
Epoch 6/50
79/79 [==============================] - 7s 85ms/step - loss: 0.0109 - accuracy: 0.9956
Epoch 7/50
79/79 [==============================] - 7s 85ms/step - loss: 0.0108 - accuracy: 0.9948
Epoch 8/50
79/79 [==============================] - 7s 85ms/step - loss: 0.0103 - accuracy: 0.9944
Epoch 9/50
79/79 [==============================] - 7s 86ms/step - loss: 0.0102 - accuracy: 0.9952
Epoch 10/50
79/79 [==============================] - 7s 84ms/step - loss: 0.0104 - accuracy: 0.9956
Epoch 11/

In [21]:
print(result.history)
print(result)

{'loss': [0.011579281650483608, 0.010895702987909317, 0.00976040679961443, 0.010581343434751034, 0.011904683895409107, 0.010861502029001713, 0.010826483368873596, 0.010318174958229065, 0.010150991380214691, 0.010435348376631737, 0.011392052285373211, 0.009685474447906017, 0.012550631538033485, 0.2619255781173706, 0.1390770822763443, 0.03931370750069618, 0.031365979462862015, 0.022514255717396736, 0.014571476727724075, 0.012526244856417179, 0.011941595003008842, 0.012418700382113457, 0.011477451771497726, 0.012071789242327213, 0.010992681607604027, 0.011076618917286396, 0.010853820480406284, 0.009794206358492374, 0.010028052143752575, 0.009630932472646236, 0.010219547897577286, 0.010813973844051361, 0.00954597070813179, 0.009756647981703281, 0.009364638477563858, 0.010903900489211082, 0.00924451369792223, 0.01020500436425209, 0.031833261251449585, 0.07404628396034241, 0.02716211974620819, 0.012304565869271755, 0.011661769822239876, 0.011423023417592049, 0.011818652041256428, 0.010543507

## Speicher das Modell

In [18]:
model.save(model_path)